In [ ]:
import requests
import base64
import json
import os # To securely load credentials
import pandas as pd

# --- Your Credentials ---
# NOTE: Replace these with your actual Client ID and Secret from your Spotify Developer Dashboard
# In a real project, you would load these from environment variables or a secure secret manager.
CLIENT_ID = '45334c0dc327460aab54a89fb94be328'
CLIENT_SECRET = 'ad8df9758b9a4e048e770a0999dfc91c'
TOKEN_URL = 'https://accounts.spotify.com/api/token'

def get_spotify_token():
    """Authenticates using the Client Credentials Flow and returns an Access Token."""
    
    # Base64 Encode the Client ID and Secret (required for the Authorization header)
    # Format: Base64(CLIENT_ID:CLIENT_SECRET)
    client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
    client_creds_b64 = base64.b64encode(client_creds.encode()).decode()
    
    # 1. Prepare Request Headers and Body
    headers = {
        "Authorization": f"Basic {client_creds_b64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials" # Required grant type for server-to-server
    }
    
    # 2. Send POST Request to Token Endpoint
    try:
        response = requests.post(TOKEN_URL, headers=headers, data=data)
        response.raise_for_status() # Raise exception for bad status codes (4xx or 5xx)
        token_data = response.json()
        
        # 3. Return the Access Token (valid for 3600 seconds, or 1 hour)
        return token_data['access_token']
    
    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error obtaining token: {err}")
        print(f"Response: {response.text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Get the token once for use in subsequent calls
SPOTIFY_TOKEN = get_spotify_token()
print(f"Spotify Token Retrieved: {'Yes' if SPOTIFY_TOKEN else 'No'}")

Spotify Token Retrieved: Yes


In [5]:
def get_artist_features(artist_name, token):
    """Searches for an artist and retrieves their popularity and follower count."""
    if not token:
        return None
        
    BASE_API_URL = 'https://api.spotify.com/v1/'
    
    # --- Step A: Search for the Artist to get the Spotify ID ---
    search_url = f"{BASE_API_URL}search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1 # Only need the top match
    }
    
    search_response = requests.get(search_url, headers=headers, params=params)
    search_response.raise_for_status()
    search_results = search_response.json()
    
    artists_list = search_results['artists']['items']
    if not artists_list:
        print(f"Artist not found: {artist_name}")
        return None
        
    artist_id = artists_list[0]['id']
    
    # --- Step B: Get Full Artist Details using the ID ---
    artist_url = f"{BASE_API_URL}artists/{artist_id}"
    
    artist_response = requests.get(artist_url, headers=headers)
    artist_response.raise_for_status()
    artist_data = artist_response.json()
    
    # --- Step C: Extract Features ---
    popularity = artist_data.get('popularity') # Score between 0 and 100
    followers = artist_data.get('followers', {}).get('total', 0)
    genres = artist_data.get('genres', [])
    
    return {
        'artist_id': artist_id,
        'popularity_score': popularity,
        'total_followers': followers,
        'genres': genres
    }

# --- Example Usage (Add this to Notebook 1) ---
if SPOTIFY_TOKEN:
    artists_to_lookup = ["The Home Team", "A Day to Remember", "Bring Me The Horizon", "Hot Mulligan"]
    artist_features_list = []
    
    for artist in artists_to_lookup:
        try:
            features = get_artist_features(artist, SPOTIFY_TOKEN)
            if features:
                artist_features_list.append({'artist_name': artist, **features})
                print(f"✅ Found {artist}: Pop={features['popularity_score']}, Followers={features['total_followers']}")
        except Exception as e:
            print(f"Error processing {artist}: {e}")

# Convert to a DataFrame for merging with your ticket data
import pandas as pd
features_df = pd.DataFrame(artist_features_list)
print("\nGenerated Feature DataFrame:")
print(features_df)

✅ Found The Home Team: Pop=64, Followers=249443
✅ Found A Day to Remember: Pop=70, Followers=2532712
✅ Found Bring Me The Horizon: Pop=79, Followers=6693008
✅ Found Hot Mulligan: Pop=64, Followers=338744

Generated Feature DataFrame:
            artist_name               artist_id  popularity_score  \
0         The Home Team  0hp58JplihFjlLstUbKS0x                64   
1     A Day to Remember  4NiJW4q9ichVqL1aUsgGAN                70   
2  Bring Me The Horizon  1Ffb6ejR6Fe5IamqA5oRUF                79   
3          Hot Mulligan  1lKZzN2d4IqiEYxyECIEHI                64   

   total_followers                                             genres  
0           249443                                         [pop punk]  
1          2532712  [metalcore, post-hardcore, pop punk, screamo, ...  
2          6693008     [metalcore, emo, screamo, rock, post-hardcore]  
3           338744                       [midwest emo, pop punk, emo]  
